In [6]:
import gmaps
import gmaps.datasets
import sys
sys.path.insert(0, "..")
%load_ext autoreload
%autoreload 2
from src.secrets import API_KEY

In [44]:
import googlemaps

In [17]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

In [1]:
from datetime import datetime
time = datetime.now()

In [2]:
type(time)

datetime.datetime

In [21]:
import pandas as pd

data = pd.read_csv("../data/ztm_Gdansk.csv", index_col=0)
data.head()

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon
0,Dąbrowa Centrum,Gdynia Dąbrowa Centrum,5.0,Gdynia,54.47317,18.46509
1,Kameliowa,Gdynia Kameliowa,5.0,Gdynia,54.47018,18.46820
2,Paprykowa,Gdynia Paprykowa,5.0,Gdynia,54.46784,18.46546
3,Szafranowa,Gdynia Szafranowa,5.0,Gdynia,54.46536,18.46020
4,Giełda Towarowa,Gdynia Giełda Towarowa,5.0,Gdynia,54.46224,18.45337


In [40]:
origins = data[data['stopDesc'] == 'Niedźwiednik'].iloc[0]
origins_coord = f"{origins.stopLat}, {origins.stopLon}"
origins_coord

'54.38, 18.55937'

In [81]:
stops_test = data.sample(20)[['stopDesc', 'stopLat', 'stopLon']].to_dict('record')
dest_coord = [f"{i['stopLat']}, {i['stopLon']}" for i  in stops_test]
dest_coord

['54.40951999999999, 18.57776',
 '54.333980000000004, 18.6158',
 '54.36385, 18.48111',
 '54.544059999999995, 18.49909',
 '54.547430000000006, 18.44199',
 '54.46816, 18.48108',
 '54.38338, 18.62058',
 '54.32241, 18.55242',
 '54.34964, 18.512439999999998',
 '54.355340000000005, 18.74535',
 '54.310919999999996, 18.62953',
 '54.53106, 18.48263',
 '54.341159999999995, 18.65822',
 '54.34936, 18.67455',
 '54.471869999999996, 18.47883',
 '54.33741, 18.616310000000002',
 '54.38298, 18.59881',
 '54.38013, 18.58689',
 '54.59495, 18.4297',
 '54.50488000000001, 18.55269']

In [82]:
gmaps_cli = googlemaps.Client(key=API_KEY)

result = gmaps_cli.distance_matrix(
                            origins=origins_coord,
                            destinations=dest_coord,
                            mode='transit',
                            units='metrics',
                            departure_time=time,
                            traffic_model='best_guess')

result

{'destination_addresses': ['Generała T. Bora-Komorowskiego 5, 80-382 Gdańsk, Poland',
  'Chełm - Więckowskiego 01, 80-766 Gdańsk, Poland',
  'Kokoszki - Poczta 01, 80-298 Gdańsk, Poland',
  'Gołębia 02, Gdynia, Poland',
  'Jęczmienna 01, 81-006 Gdynia, Poland',
  'Górnicza - Damroki 02, 81-572 Gdynia, Poland',
  'Mickiewicza 01., 80-425 Gdańsk, Poland',
  'Czermińskiego 01, 80-180 Gdańsk, Poland',
  'Karczemki 01, 80-177 Gdańsk, Poland',
  'Górecka 01, 80-644 Gdańsk, Poland',
  'Ukośna 02, 80-058 Gdańsk, Poland',
  'Ściegiennego 01, 81-200 Gdynia, Poland',
  'Śluza 01, 80-700 Gdańsk, Poland',
  'Mostek 01, 80-761 Gdańsk, Poland',
  'Wiczlińska 02, 81-589 Gdynia, Poland',
  'Chełm Witosa 04, 80-809 Gdańsk, Poland',
  'Galeria Bałtycka 05, 80-265 Gdańsk, Poland',
  'Chrzanowskiego 02, 80-257 Gdańsk, Poland',
  'Kazimierz - Majowa 01, 84-230, Poland',
  'Ejsmonda 02, 81-409 Gdynia, Poland'],
 'origin_addresses': ['Niedźwiednik 01, 80-292 Gdańsk, Poland'],
 'rows': [{'elements': [{'distanc

In [83]:
travel_times = [i['duration']['value'] for i in result['rows'][0]['elements']]

In [84]:
lat_stops = [float(i.split(', ')[0]) for i in dest_coord]
lon_stops = [float(i.split(', ')[1]) for i in dest_coord]

In [85]:
origin_lat = float(origins_coord.split(', ')[0])
origin_lon = float(origins_coord.split(', ')[1])

In [86]:
"""
      t < 15  green
15 <= t < 30  blue
45 <= t < 60  yellow
60 <= t < 90  red
90 <= t       black
"""

'\n      t < 15  green\n15 <= t < 30  blue\n45 <= t < 60  yellow\n60 <= t < 90  red\n90 <= t       black\n'

In [87]:
def times_to_colors(t):
    if t < 900:
        color = "green"
    elif 900 <= t < 1800:
        color = "blue"
    elif 1800 <= t < 2700:
        color = "yellow"
    elif 2700 <= t < 3600:
        color = "red"
    else:
        color = "black"
    return color

In [88]:
destination_times = pd.DataFrame({'lat': lat_stops, 'lon': lon_stops, 'time': travel_times})

In [89]:
destination_times['color'] = destination_times['time'].apply(lambda x: times_to_colors(x))

In [100]:
destination_times

,lat,lon,time,color
0,54.40952,18.57776,3303,red
1,54.33398,18.61580,2519,yellow
2,54.36385,18.48111,2295,yellow
3,54.54406,18.49909,4873,black
4,54.54743,18.44199,4914,black
5,54.46816,18.48108,5016,black
6,54.38338,18.62058,3060,red
7,54.32241,18.55242,2851,red
8,54.34964,18.51244,2907,red
9,54.35534,18.74535,4800,black


In [108]:
output_file("gmap.html")

map_options = GMapOptions(lat=origin_lat, lng=origin_lon, map_type="roadmap", zoom=11)

p = gmap(API_KEY, map_options, title="3city")

source = ColumnDataSource(
                    data=dict(lat=destination_times.lat.to_list(),
                              lon=destination_times.lon.to_list(),
                              color=destination_times.color.to_list())
)
origin = ColumnDataSource(
                    data=dict(lat=[origin_lat],
                              lon=[origin_lon])
)

p.circle(x="lon", y="lat", size=15, fill_color="color", fill_alpha=0.8, source=source)
p.circle(x="lon", y="lat", size=20, fill_color="grey", fill_alpha=0.8, source=origin)

show(p)

In [104]:
source = ColumnDataSource(
                    data=dict(lat=destination_times.lat.to_list(),
                              lon=destination_times.lon.to_list(),
                              color=destination_times.color.to_list()))

In [107]:
source.data["color"]

['red',
 'yellow',
 'yellow',
 'black',
 'black',
 'black',
 'red',
 'red',
 'red',
 'black',
 'black',
 'black',
 'red',
 'yellow',
 'black',
 'yellow',
 'blue',
 'blue',
 'black',
 'black']